# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [38]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
# Load the csv
weather_csv_file = "output_data/city_weather_data.csv"
weather_df = pd.read_csv(weather_csv_file)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.8000,-68.3000,31.66,93.0,40.0,20.71,AR,1.631777e+09
1,victoria,22.2855,114.1577,93.16,72.0,39.0,1.01,HK,1.631777e+09
2,sao filipe,14.8961,-24.4956,79.59,81.0,25.0,9.80,CV,1.631777e+09
3,okhotsk,59.3833,143.3000,57.09,76.0,99.0,10.11,RU,1.631777e+09
4,ahipara,-35.1667,173.1667,58.95,97.0,67.0,6.89,NZ,1.631777e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Change type of Humidity to float
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations and change to float
locations = weather_df[["Lat", "Lng"]].astype(float)

# Create the humidity Heatmap layer
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create the heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
# Add the heat layer 
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# Narrow down the DataFrame to find your ideal weather condition
#  70(F)< max temperature between< 80(F)
# Wind speed less < 10 mph
# cloudiness=0

ideal_weather_df = weather_df.loc[(weather_df["Max Temp"] >= 70) &
                                  (weather_df["Max Temp"] <= 80) &
                                  (weather_df["Wind Speed"] <= 10) &
                                  (weather_df["Cloudiness"] == 0)].dropna()

ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,benghazi,32.1167,20.0667,78.91,56.0,0.0,8.37,LY,1.631777e+09
265,bom jesus,-9.0744,-44.3586,72.10,45.0,0.0,3.06,BR,1.631777e+09
328,yumen,40.2833,97.2000,78.31,18.0,0.0,7.34,CN,1.631777e+09
344,belyy yar,53.6039,91.3903,71.49,37.0,0.0,4.47,RU,1.631777e+09
346,jacareacanga,-6.2222,-57.7528,71.19,83.0,0.0,1.41,BR,1.631777e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = ideal_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No results found...Skipping")
        
    print("------------")

Retrieving Results for Index 24: benghazi.
Closest hotel is Tibesti Hotel.
------------
Retrieving Results for Index 265: bom jesus.
Closest hotel is Hotel Brasão & Floricultura Bella Flor.
------------
Retrieving Results for Index 328: yumen.
No results found...Skipping
------------
Retrieving Results for Index 344: belyy yar.
Closest hotel is Pasportnyy Stol.
------------
Retrieving Results for Index 346: jacareacanga.
Closest hotel is Casa do Ian😎.
------------
Retrieving Results for Index 357: mildura.
Closest hotel is Mercure Hotel Mildura.
------------
Retrieving Results for Index 358: huercal-overa.
Closest hotel is Hotel Overa S L.
------------
Retrieving Results for Index 379: caravelas.
Closest hotel is Pousada dos Navegantes.
------------
Retrieving Results for Index 382: campo grande.
Closest hotel is Hotel Harbor Self Buriti Suítes Campo Grande.
------------
Retrieving Results for Index 427: broken hill.
Closest hotel is Royal Exchange Hotel.
------------
Retrieving Result

In [46]:
# Display the dataframe
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,benghazi,32.1167,20.0667,78.91,56.0,0.0,8.37,LY,1.631777e+09,Tibesti Hotel
265,bom jesus,-9.0744,-44.3586,72.10,45.0,0.0,3.06,BR,1.631777e+09,Hotel Brasão & Floricultura Bella Flor
328,yumen,40.2833,97.2000,78.31,18.0,0.0,7.34,CN,1.631777e+09,NaN
344,belyy yar,53.6039,91.3903,71.49,37.0,0.0,4.47,RU,1.631777e+09,Pasportnyy Stol
346,jacareacanga,-6.2222,-57.7528,71.19,83.0,0.0,1.41,BR,1.631777e+09,Casa do Ian😎


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))